In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
db name - hop_db
table names - hop / npidata / taxonomy/ zip_to_cbsa

In [2]:
# Create the hop_team SQLite database
db = sqlite3.connect('hop_db.sqlite')

In [ ]:
# Create the hop_team SQLite database
db = sqlite3.connect('hop_db.sqlite')

# create the referrals table and add it to the database
for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
    # Do some preemptive filtering for likely "accidental" referrals
    chunk = chunk.loc[(chunk['transaction_count'] >= 25) & (chunk['std_day_wait'] < 90)]
    # append to referrals table
    chunk.to_sql('hop', db, if_exists = 'append', index = False)

In [10]:
taxonomy = pd.read_csv('nucc_taxonomy_230.csv', encoding = 'latin1')

In [11]:
taxonomy.to_sql('taxonomy', db, if_exists = 'append', index = False)

873

In [12]:
zip_cbsa = pd.read_excel('ZIP_CBSA_122021.xlsx')

In [14]:
zip_cbsa.to_sql('zip_to_cbsa', db, if_exists = 'append', index = False)

47484

In [ ]:
# Old code but may use later on

# Monica's magic loop!
def get_primary_taxonomy(row):
    for n in range(1, 16):
        if row['primary_taxonomy'] == '':
            x = str(n)
            if row[f'Healthcare Provider Primary Taxonomy Switch_{x}'] == 'Y':
                return row[f'Healthcare Provider Taxonomy Code_{x}']
    return row['primary_taxonomy']

# Define list of relevant columns
cols = ['NPI', 'Entity Type Code', 
        'Provider Organization Name (Legal Business Name)',
        'Provider Last Name (Legal Name)', 'Provider First Name', 
        'Provider Middle Name', 'Provider Name Prefix Text',
        'Provider Name Suffix Text', 'Provider Credential Text',
        'Provider First Line Business Practice Location Address',
        'Provider Second Line Business Practice Location Address',
        'Provider Business Practice Location Address City Name',
        'Provider Business Practice Location Address State Name',
        'Provider Business Practice Location Address Postal Code']

# Append taxonomy columns 1 to 15
for i in range(1, 16):
    cols.append(f'Healthcare Provider Primary Taxonomy Switch_{i}')
    cols.append(f'Healthcare Provider Taxonomy Code_{i}')

# Read in CSV and relevant columns in chunks
chunksize = 1000000
npi_chunks = pd.read_csv('npidata_pfile_20050523-20230212.csv', usecols = cols, low_memory = False, chunksize = chunksize)

# Define empty dataframe
npi_test = pd.DataFrame()

# Loop over the chunks and process the data
for chunk in npi_chunks:
    # Process data with apply function
    chunk['primary_taxonomy'] = ''
    chunk['primary_taxonomy'] = chunk.apply(get_primary_taxonomy, axis = 1)

    # Append processed data to the empty dataframe
    npi_test = pd.concat([npi_test, chunk])

# Read in other csv files with type cleaning    
nucc = pd.read_csv('nucc_taxonomy_230.csv', encoding = 'latin1')
zip = pd.read_csv('ZIP_CBSA_122021.csv', dtype = {'zip': str})
zip['cbsa'] = zip['cbsa'].astype(str)

# Merge and edit zip codes
merged_df = pd.merge(npi_test, nucc[['Code', 'Classification']], left_on = 'primary_taxonomy', right_on = 'Code', how = 'left')
merged_df['Provider Business Practice Location Address Postal Code'] = merged_df['Provider Business Practice Location Address Postal Code'].str[:5]
merged_df = pd.merge(merged_df, zip[['zip', 'cbsa']], left_on = 'Provider Business Practice Location Address Postal Code', right_on = 'zip', how = 'left')

merged_df.tail(25)

In [3]:
query = """
SELECT n.npi, n.organization_name
FROM npidata n
INNER JOIN taxonomy t 
ON n.taxonomy_code = t.Code
WHERE n.entity_type_code = 2
  AND n.practice_location_address_city_name = 'NASHVILLE'
  AND n.practice_location_address_state_name = 'TN'
  AND t.classification = 'General Acute Care Hospital'
"""

with sqlite3.connect('hop_db.sqlite') as db: 
    hospitals = pd.read_sql(query, db)
    
db.close()
hospitals

,npi,organization_name
0,1609056399,VANDERBILT CHILDREN'S
1,1326224015,VANDERBILT CHILDRENS HOSPITAL
2,1609970029,"BUCKS COUNTY ONCOPLASTIC INSTITUTE, LLC"
3,1164609319,VANDERBILT MEDICAL CENTER
4,1992861314,SETON CORPORATION
5,1477709053,VANDERBILT UNIVERSITY MEDICAL CENTER
6,1255577466,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL
7,1396983680,"ST. THOMAS NP, LLC"
8,1366679276,VANDERBILT UNIVERSITY MEDICAL CENTER
9,1518112440,NASHVILLE REHAB LLC


In [ ]:
query = '''
SELECT *
FROM hop
INNER JOIN npidata n1
ON hop.from_npi = n1.npi
INNER JOIN npidata n2
ON hop.to_npi = n2.npi
WHERE n1.entity_type_code = 1
    AND n2.practice_location_address_city_name = 'NASHVILLE'
    AND n2.practice_location_address_state_name = 'TN'
    AND n2.entity_type_code = 2;
'''
with sqlite3.connect('hop_db.sqlite') as db: 
    total_nashville = pd.read_sql(query, db)
    
db.close()
total_nashville